In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# A. Acquisition

|          | Description                                                  |         Amount |
| -------- | ------------------------------------------------------------ | -------------: |
| nodes    | mice                                                         |      100 - 200 |
| edges    | similar genes, protein expressions, or phenotypes            | O(10) per node |
| features | genes, protein expressions in tissues, or phenotypes         |          1000s |
| labels   | depends: a particular gene, phenotype, or protein expression |            N/A |

## A.0 Loading

### genotype_BXD

In [ ]:
genotype_df = pd.read_csv("data/genotype_BXD.txt", sep='\t', index_col='SNP')

In [ ]:
genotype_df.shape

In [ ]:
genotype_df = genotype_df.transpose()

genotype_df.index.name = 'BXD_strain'

In [ ]:
print("There is NaN values: %s" % genotype_df.isna().any().any())

In [ ]:
np.unique(genotype_df.values, return_counts=True)

In [ ]:
genotype_df.head()

<div class="alert alert-block alert-info">
    elements are set to -1, 0, and 1 for the homozygote, heterozygote, and other homozygote (AA, (Aa, aA), aa)
</div>

### Phenotype

In [ ]:
phenotype_df = pd.read_csv("data/Phenotype.txt", sep='\t', index_col='PhenoID')
phenotype_df.head()

In [ ]:
phenotype_df.shape

In [ ]:
nan_count = phenotype_df.isna().sum().sum()
entries_count = phenotype_df.shape[0] * phenotype_df.shape[1]

print("Number of Nan values: %s" % nan_count)
print("Percentage of nan in the phenotype file: {:0.2f}%".format(nan_count / entries_count * 100))

### Protein Expression

## A.1 Cleaning

### Genotype

genotype_df is already clean


In [ ]:
genotype_df.to_pickle("data/pickle/genotype.pkl")

## A.2 Building graph from features (Preprocessing)

In [ ]:
from scipy.spatial.distance import pdist, squareform

### Genotype graph (gene based similarity)

To build our first graph, we chose as nodes the mice and as edges a gene based similarity

In [ ]:
genotype_df = pd.read_pickle("data/pickle/genotype.pkl")

In [ ]:
strain_genetic_dist = pdist(genotype_df.values, metric='euclidean')

Check the mean pairwise distance  𝔼[𝐷]

In [ ]:
from scipy.stats import norm
import scipy
import matplotlib
def hist_norm_fit(serie: pd.Series, ax: matplotlib.axes.Axes, meth: scipy.stats, bins=30):
    """
    performs a histogram of a given series and fits a normal pdf to it
    parameters:
        serie: the data series to plot
        ax: the axis to plot on
        bins: the number of bins for the histograms
        meth: name of the pdf to fit to the data (from scipy)
    """
    # Fit a normal distribution to the data:
    if serie.isnull().values.any():
        mu = np.nanmean(serie.values)
        std = np.nanstd(serie.values)
    else:
        mu, std = meth.fit(serie.values)
    

    # Plot the histogram.
    #dataset.Overall.hist(ax = ax,bins = 30)
    ax.hist(serie, bins=bins, density=True, alpha=0.6, color='m')

    # Plot the PDF.
    xt = ax.get_xticks()
    xmin, xmax = np.min(xt),np.max(xt)

    x = np.linspace(xmin, xmax, 100)
    if meth is norm:
        p = meth.pdf(x, mu, std)
    else:
        p = meth.pdf(x)
    ax.plot(x, p, 'k', linewidth=2)
    title = "Fit results to normal: $\mu$ = %.2f,  $\sigma$ = %.2f" % (mu, std)
    ax.set_title(title)

In [ ]:

ax = plt.subplot()
plt.suptitle("Histogram of Euclidean distances between mice strains", fontweight='bold')
strain_gen_dist_series = pd.Series(strain_genetic_dist.flatten())
hist_norm_fit(strain_gen_dist_series, ax, meth=norm, bins=100)
ax.set_xlabel("genetic distance")
plt.show()

In [ ]:
ax = plt.subplot()
plt.suptitle("Histogram of Euclidean distances between mice strains", fontweight='bold')

over90_mask = (strain_gen_dist_series > 90)
hist_norm_fit(strain_gen_dist_series[over90_mask], ax, meth=norm, bins=100)

ax.set_xlabel("genetic distance")
plt.show()

<div class="alert alert-block alert-info">
    Once we mask the peak below 90, the distances genetic distance between strains looks like a gaussian distribution. Still need to investigate where does this ~90 pick comes from.
</div>

In [ ]:
# Here we take mean distance value when we don't mask over 90
mean_dist = strain_genetic_dist.mean()
std_dist = strain_genetic_dist.std()
print("Mean pairwise distance: %0.2f (+/- %0.2f)" % (strain_genetic_dist.mean(), strain_genetic_dist.std()))

Let's create adjacency matrix for the strains by thresholding the Euclidean distance matrix.
The resulting **unweighted** adjacency matrix should have entries


$$ A_{ij} = \begin{cases} 1, \; \text{if} \; d(i,j)< \mathbb{E}[D], \; i \neq j, \\ 0, \; \text{otherwise.} \end{cases} $$


In [ ]:
gen_related_strain_A.shape

In [ ]:
#Algorithm to create the edges
threshold = mean_dist
gen_related_strain_A = squareform(strain_genetic_dist).copy()
gen_related_strain_A[gen_related_strain_A < threshold] = 1
gen_related_strain_A[gen_related_strain_A >= threshold] = 0
gen_related_strain_A -= np.identity(gen_related_strain_A.shape[0])
gen_related_strain_A.shape

In [ ]:
plt.figure(figsize=(6, 6))
plt.spy(gen_related_strain_A)
plt.suptitle("Adj. matrix for genetic relation between strains", fontweight='bold')
plt.show()

In [ ]:
np.save("data/numpy/gen_related_strain_A.npy", gen_related_strain_A)

# B. Exploration

# C. Exploitation